<a href="https://colab.research.google.com/github/arangodb/interactive_tutorials/blob/master/Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook provides explanation on how to use this template, if you would like the '**quick start**' version without any explanation, you can find that [here]().

# Getting Started with ArangoDB Interactive Notebooks

This notebook provides instructions and examples for creating interactive tutorial notebooks for ArangoDB. If you have a cool project you would like to share or a concept that you think the community might enjoy, please create a notebook and submit a PR.

This notebook covers:
* How to generate a free temporary database
* Installing and using the python driver 
* Importing data 

## Getting Connected

### Temporary database
Our notebooks use a Foxx microservice to generate a temporary database and you can use it as well. 

#### Hosted for Free
We offer the endpoint hosted on Oasis to use for free, the URL to use for that is https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB. 

When making a notebook be sure to include `tutorialName`(as shown below) if you use the endpoint.

#### Local
Alternatively, you can test and create your own notebooks using the foxx service locally by simply zipping the `foxx_service` directory in the root of the [interactive-tutorials repository](https://github.com/arangodb/interactive_tutorials) and installing in your local ArangoDB instance.

## Oasis.py
The `oasis.py` package is used to manage the session and helps to ensure the database is ready before proceeding with opening any connections.

### Python Driver
There are two python drivers created and maintained by our wonderful community and you can use the one your prefer. The two most popular python drivers are:
* [pyArango](https://github.com/ArangoDB-Community/pyArango)
* [python-arango](https://github.com/joowani/python-arango)

By default `oasis.py` uses pyArango but as demonstrated below you can also use python-arango by using `oasis.connect_python_arango()` instead of the default `oasis.connect()`. 
<hr />

## Installation

Start with installing any necessary packages for your example, including your python driver of choice.

In [ ]:
%%capture
!pip3 install pyarango
!pip3 install "python-arango>=5.0"

Here we clone the `oasis.py` package along with some sample data from the `oasis_with_sample_data` branch. Typically, you will use the `oasis_connector` branch which does not include the extra sample data.

In [ ]:
%%capture
# Typically you will use -b oasis_connector which does not include the sample data
!git clone https://github.com/arangodb/interactive_tutorials.git -b oasis_with_sample_data --single-branch

!rsync -av interactive_tutorials/ ./ --exclude=.git

Here we import the oasis package along with our python driver. Here we have imported both but only one is necessary.

In [ ]:
import oasis

from pyArango.connection import *
from arango import ArangoClient

## Connecting

Be sure to update the `tutorialName` variable with your tutorials name.

In [ ]:
# Retrieve tmp credentials from ArangoDB Tutorial Service

# ** UPDATE THE FOLLOWING VARIABLE  **
tutorialName = "YOUR AWESOME TUTORIAL NAME GOES HERE"
login = oasis.getTempCredentials(tutorialName=tutorialName, credentialProvider="https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB")

# Here is an example of connecting with the default pyArango driver
connection = oasis.connect(login)
db = connection[login["dbName"]]

# Here is an example of connecting with python arango 
# database = oasis.connect_python_arango(login)

# These are the credentials
print("https://"+login["hostname"]+":"+str(login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

# Importing Data

If your tutorial relies on some external data you will need a way to load it into the newly created database. One way to achieve this is to copy it from somewhere such as GitHub, like we did earlier:

```
%%capture
!git clone https://github.com/arangodb/interactive_tutorials.git -b oasis_connector --single-branch
!rsync -av interactive_tutorials/ ./ --exclude=.git
```
First we cloned the single oasis_connector branch and then used rsync to save into our working directory. 

Once you have the data you need a way to get it into your newly created database. You are free to parse and import your data however you choose but some simple options for those already familiar with ArangoDB are:
* [arangorestore](https://www.arangodb.com/docs/stable/programs-arangorestore.html)
* [arangoimport](https://www.arangodb.com/docs/stable/programs-arangoimport.html)

Here we confirm that the tools directory can be read and executed. The tools directory is where we store the Linux binaries for the vairous import tools.

In [ ]:
!chmod -R 755 ./tools/*

## arangorestore

[Arangorestore](https://www.arangodb.com/docs/stable/programs-arangorestore.html) is a command-line client tool to restore backups created by Arangodump to ArangoDB servers.

Here is an example of loading the ArangoDB dump of the imdb dataset.

**Note**: We are using the `login` variable which contains the newly created database credentials.

In [ ]:
# Extra precaution for this notebook, drop all collections to avoid naming conflicts.
# You typically will not need to drop all collections.
db.reloadCollections()
if db.hasCollection('imdb_vertices') or db.hasCollection('imdb_edges'):
   db.dropAllCollections()

# Restore an ArangoDB dump into the database.
! ./tools/arangorestore \
-c none \
--server.endpoint http+ssl://{login["hostname"]}:{login["port"]} \
--server.username {login["username"]} \
--server.database {login["dbName"]} \
--server.password {login["password"]} \
--replication-factor 3  \
--input-directory "data/imdb_dump" 

## arangoimport
[Arangoimport](https://www.arangodb.com/docs/stable/programs-arangoimport.html) is a command-line client tool to import data in JSON, CSV and TSV format to ArangoDB servers.

Here is an example of loading the imdb dataset from JSON files.

Notes:
* We are using the `login` variable which contains the newly created database credentials.
* Make sure the `--type` option matches your file type.
* You may need to adjust `--batch-size` for  your file
* Note that we specify `--create-collection-type` to `edge` 

In [ ]:
# Extra precaution for this notebook, drop all collections to avoid naming conflicts.
# You typically will not need to drop all collections.
db.reloadCollections()
if db.hasCollection('imdb_vertices') or db.hasCollection('imdb_edges'):
   db.dropAllCollections()

# Import data from CSV, TSV, or JSON into the database. (Document Colleciton)
! ./tools/arangoimport \
-c none \
--server.endpoint http+ssl://{login["hostname"]}:{login["port"]} \
--server.username {login["username"]} \
--server.database {login["dbName"]} \
--server.password {login["password"]} \
--file "data/vertices/imdb_vertices.json" \
--type json \
--collection "imdb_vertices" \
--create-collection true \
--batch-size 44048576

# Import data from CSV, TSV, or JSON into the database. (Edge Colleciton)
! ./tools/arangoimport \
-c none \
--server.endpoint http+ssl://{login["hostname"]}:{login["port"]} \
--server.username {login["username"]} \
--server.database {login["dbName"]} \
--server.password {login["password"]} \
--file "data/edges/imdb_edges.json" \
--type json \
--collection "imdb_edges" \
--create-collection true \
--create-collection-type edge \
--batch-size 44048576

You now have a tutorial database with your data loaded and the sky is the limit!

If you would like to share your notebook simply place it in the `community_notebooks` folder in the [interactive-tutorials repository](https://github.com/arangodb/interactive_tutorials) and make a pull request.

Good luck and we are excited to see what you are working on!